In [1]:
# Import packages
import pandas as pd
import plotly.graph_objects as go

In [2]:
# Import data
df = pd.read_excel('Monthly Financial Results 1998-present.xlsx')
df['month'] = df['month_ended'].dt.to_period('M').dt.to_timestamp()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Financial Year        303 non-null    object        
 1   File name             303 non-null    object        
 2   month_ended           303 non-null    datetime64[ns]
 3   revenue               303 non-null    float64       
 4   expenditure           303 non-null    float64       
 5   surplus_deficit       303 non-null    float64       
 6   proceed_repayment     17 non-null     float64       
 7   fiscal_reserve_start  303 non-null    float64       
 8   fiscal_reserve_end    303 non-null    float64       
 9   D+E-F                 303 non-null    float64       
 10  F+G+H-I               303 non-null    int64         
 11  start - previous end  302 non-null    float64       
 12  month                 303 non-null    datetime64[ns]
dtypes: datetime64[ns](2)

In [3]:
# Hong Kong Fiscal Reserves 1998-Present
df['measure'] = 'relative'
df['change_billion'] = df['fiscal_reserve_end'].diff() / 1000
df['change_billion'][0] = (df['fiscal_reserve_end'][0] - df['fiscal_reserve_start'][0]) / 1000
df['surplus_or_deficit'] = df['change_billion'].apply(lambda x: 'Surplus' if x >= 0 else 'Deficit')

fig = go.Figure(go.Waterfall(
    measure = ['relative'] + df['measure'].tolist(),
    x = [pd.to_datetime('1998-08-01')] + df['month'].tolist() ,
    y = [df['fiscal_reserve_start'][0]/1000] + df['change_billion'].tolist(),
    text = ['Opening balance'] + df['surplus_or_deficit'].tolist(),
    width = 1500000000,
    increasing_marker_color = '#31CAA8',
    decreasing_marker_color = '#FF412C',
    texttemplate = ' ',
    hovertemplate = 'Start: HK$%{initial:,.1f} billion<br>%{text}: HK$%{delta:,.1f} billion<br>End: HK$%{final:,.1f} billion<extra></extra>'
))

fig.update_layout(
    title = "Hong Kong Fiscal Reserves 1998-Present",
    showlegend = False,
    margin = dict(t=90, b=30, l=80, r=80),
    hovermode = 'x unified',
    template = 'plotly_white',
    paper_bgcolor = '#eaeaea',
    plot_bgcolor = '#eaeaea',
    hoverlabel = dict(
        bgcolor = '#eaeaea'
    )

)

fig.update_xaxes(
    linecolor = '#d6d6d6',
    hoverformat = '%B %Y',
    rangeselector= dict(
        buttons = list([
            dict(count=1,
                label="1y",
                step="year",
                stepmode="backward"),
            dict(count=5,
                label="5y",
                step="year",
                stepmode="backward"),
            dict(count=10,
                label="10y",
                step="year",
                stepmode="backward"),
            dict(step="all")
        ]),
        bgcolor = '#eaeaea',
        x = 0,
        y = 1
    ),
)

fig.update_yaxes(
    tickformat = ',',
    gridcolor = '#d6d6d6'
)

fig.add_annotation(
    xref = 'paper',
    x = 0,
    yref = 'paper',
    y = 1,
    text = 'HK$ billion',
    showarrow = False,
    xanchor = 'right',
    yanchor = 'middle'
)

fig.show()

fig.write_html('waterfall.html')

/var/folders/n9/689517md5nbd3sqsdnk9mgd00000gn/T/ipykernel_4183/1511207874.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['change_billion'][0] = (df['fiscal_reserve_end'][0] - df['fiscal_reserve_start'][0]) / 1000


In [4]:
# Hong Kong Fiscal Surplus / Deficit 1998-Present
df['revenue_billion'] = df['revenue'] / 1000
df['expenditure_billion'] = df['expenditure'] / 1000
df['proceed_repayment_billion'] = df['proceed_repayment'] / 1000
df['surplus_deficit_billion'] = df['revenue_billion'] + df['expenditure_billion'] + df['proceed_repayment_billion'].fillna(0)

fig = go.Figure()

def add_bar(data, name, color):
    fig.add_trace(
        go.Bar(
            name = name,
            x = df['month'],
            y = df[data],
            marker_color = color,
            marker_line_width = 0,
            opacity = 0.6,
            hovertemplate = 'HK$%{y:,.1f} billion'
        )
    )

add_bar('revenue_billion', 'Revenue', '#31CAA8')
add_bar('expenditure_billion', 'Expenditure', '#FF412C')
add_bar('proceed_repayment_billion', 'Bonds & notes', '#FFD13C')

fig.add_trace(
    go.Scatter(
        name = 'Surplus/Deficit',
        x = df['month'],
        y = df['surplus_deficit_billion'],
        mode= 'lines',
        line_color = '#1770B8',
        opacity = 0.9,
        hovertemplate = 'HK$%{y:,.1f} billion'
    )
)

fig.update_layout(
    title = "Hong Kong Fiscal Surplus/Deficit 1998-Present",
    showlegend = True,
    legend_orientation = 'h',
    margin = dict(t=90, b=30, l=80, r=80),
    barmode = 'relative',
    hovermode = 'x unified',
    template = 'plotly_white',
    paper_bgcolor = '#eaeaea',
    plot_bgcolor = '#eaeaea',
    hoverlabel = dict(
        bgcolor = '#eaeaea'
    ),
)

fig.update_xaxes(
    linecolor = '#d6d6d6',
    hoverformat = '%B %Y',
    rangeselector= dict(
        buttons = list([
            dict(count=1,
                label="1y",
                step="year",
                stepmode="backward"),
            dict(count=5,
                label="5y",
                step="year",
                stepmode="backward"),
            dict(count=10,
                label="10y",
                step="year",
                stepmode="backward"),
            dict(step="all")
        ]),
        bgcolor = '#eaeaea',
        x = 0,
        y = 1
    ),
)

fig.update_yaxes(
    tickformat = ',',
    gridcolor = '#d6d6d6'
)

fig.add_annotation(
    xref = 'paper',
    x = 0,
    yref = 'paper',
    y = 1,
    text = 'HK$ billion',
    showarrow = False,
    xanchor = 'right',
    yanchor = 'middle'
)

fig.show()

fig.write_html('surplus-deficit.html')